<a href="https://colab.research.google.com/github/Svetorus/Spark/blob/main/Spark_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 53 kB/s 
     |████████████████████████████████| 199 kB 80.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=7fc8fd0ceffeff588efd460fcc97b2aabb8b99d59a12d91cab3e73c3d394b218
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [4]:
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [6]:
!pip install -q findspark

In [7]:
import findspark
findspark.init()

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [9]:
# библиотеки
import pandas as pd
import numpy as np
from google.colab import files

In [10]:
# загрузка из компа в google colab
uploaded = files.upload()

Saving owid-covid-data.csv to owid-covid-data.csv


In [11]:
# библиотеки SPARK
import pyspark
df_spark = spark.read.option('interSchema','true').option('header','true').csv('owid-covid-data.csv')
# df = spark.read.csv('owid-covid-data.csv',header=True)

from pyspark.sql.functions import col,lit,lag,round,max
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as func
from pyspark.sql.window import Window
from pyspark.sql import functions as F

In [ ]:
# df_spark.printSchema()

# *Pandas*

In [13]:
covid = pd.read_csv('owid-covid-data.csv')
covid.head(3)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511


In [ ]:
covid.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'new_vaccinations',
       'new_vaccinations_smoothed', 'total_vaccinations_per_hun

# 1. Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)  
### *Pandas*

In [ ]:
covid['recovered'] = covid['total_cases'] * 100 / covid['population']
covid['recovered'] = covid['recovered'].round(2)
covid[['iso_code','location','recovered']].loc[covid['date']=='2021-03-31',]\
  .nlargest(15, 'recovered')

,iso_code,location,recovered
2079,AND,Andorra,15.54
49948,MNE,Montenegro,14.52
19623,CZE,Czechia,14.31
63600,SMR,San Marino,13.94
67304,SVN,Slovenia,10.37
44270,LUX,Luxembourg,9.85
36842,ISR,Israel,9.63
78438,USA,United States,9.20
65221,SRB,Serbia,8.83
6376,BHR,Bahrain,8.49


### PySpark

In [ ]:
df_spark.filter( (col('date') == lit('2021-03-31')) )\
  .withColumn("int_total_cases", col("total_cases").cast(IntegerType()))\
  .withColumn("int_population", col("population").cast(IntegerType()))\
  .withColumn("recovered", (col('int_total_cases') * 100 / col('int_population')))\
  .select('iso_code','location',(round('recovered', 2)).alias('recovered')).orderBy(col("recovered").desc()).show(15)

+--------+----------+---------+
|iso_code|  location|recovered|
+--------+----------+---------+
|     AND|   Andorra|    15.54|
|     MNE|Montenegro|    14.52|
|     CZE|   Czechia|    14.31|
|     SMR|San Marino|    13.94|
|     SVN|  Slovenia|    10.37|
|     LUX|Luxembourg|     9.85|
|     ISR|    Israel|     9.63|
|     SRB|    Serbia|     8.83|
|     BHR|   Bahrain|     8.49|
|     PAN|    Panama|     8.23|
|     PRT|  Portugal|     8.06|
|     EST|   Estonia|     8.02|
|     SWE|    Sweden|     7.97|
|     LTU| Lithuania|     7.94|
|     BEL|   Belgium|     7.61|
+--------+----------+---------+
only showing top 15 rows



## SQL spark

In [19]:
# загрузка данных для кода на SQL
df_spark.createOrReplaceTempView("df_spark")

In [ ]:
spark.sql('''select 
                iso_code 
                ,location 
                ,round(total_cases * 100 /population, 2 ) as recovered
            from df_spark
            where date = '2021-03-31'
            order by recovered desc; ''').show(15)

+--------+-------------+---------+
|iso_code|     location|recovered|
+--------+-------------+---------+
|     AND|      Andorra|    15.54|
|     MNE|   Montenegro|    14.52|
|     CZE|      Czechia|    14.31|
|     SMR|   San Marino|    13.94|
|     SVN|     Slovenia|    10.37|
|     LUX|   Luxembourg|     9.85|
|     ISR|       Israel|     9.63|
|     USA|United States|      9.2|
|     SRB|       Serbia|     8.83|
|     BHR|      Bahrain|     8.49|
|     PAN|       Panama|     8.23|
|     PRT|     Portugal|     8.06|
|     EST|      Estonia|     8.02|
|     SWE|       Sweden|     7.97|
|     LTU|    Lithuania|     7.94|
+--------+-------------+---------+
only showing top 15 rows



# 2. Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию
(в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)  
### *Pandas*

In [ ]:
# covid= covid[~covid['location'].isin(['World','Europe','European Union','Asia','South America','North America'])]
# covid.loc[(covid['date']<='2021-03-31') & (covid['date']>='2021-03-29')].groupby(['location'],as_index=False)\
#   .aggregate({'new_cases':'max'}).nlargest(10, 'new_cases')

In [ ]:
df= covid.loc[(covid['date']>='2021-03-24') & (covid['date']<='2021-03-31')]\
    .groupby(['location'],as_index=False).aggregate({'new_cases':'max'})
covid_2 = covid[['date','location','new_cases']].loc[(covid['date']>='2021-03-24') & (covid['date']<='2021-03-31')]
new_covid = covid_2.merge( df,  how='inner', on=['location','new_cases'])
new_covid = new_covid[~new_covid['location'].isin(['World','Europe','European Union','Asia','South America','North America','Africa'])]
new_covid.nlargest(10, 'new_cases')
 # .filter(col('location').rlike('^[^World|^Europe|^European Union|^Asia|^South America|^North America]+$'))\

,date,location,new_cases
43,2021-03-25,Brazil,100158.0
377,2021-03-24,United States,86960.0
159,2021-03-31,India,72330.0
111,2021-03-24,France,65392.0
366,2021-03-31,Turkey,39302.0
295,2021-03-26,Poland,35145.0
115,2021-03-31,Germany,25014.0
181,2021-03-26,Italy,24076.0
293,2021-03-25,Peru,19206.0
374,2021-03-26,Ukraine,18226.0


### PySpark

In [ ]:
# df_1 = df_spark.filter( (col('date') >= '2021-03-24') & (col('date') <= lit('2021-03-31')) )\
#   .filter(~F.col("location").isin(['World','Europe','European Union','Asia','South America','North America','Africa']))\
#   .withColumn("new", col("new_cases").cast(IntegerType()))
# df_1.withColumn('new_cases', F.max('new').over(Window.partitionBy('location').orderBy('location')))\
#   .distinct().orderBy(col('new_cases').desc()).select('date','location','new_cases').show(10)

In [ ]:
df_1 = df_spark.filter( (col('date') >= lit('2021-03-24')) & (col('date') <= lit('2021-03-31')) )\
  .filter(~F.col("location").isin(['World','Europe','European Union','Asia','South America','North America','Africa']))\
  .withColumn("new_cases", col("new_cases").cast(IntegerType()))\
  .groupby('location').agg(max("new_cases").alias("new_cases"))

covid_2 = df_spark.select('date','location','new_cases')\
  .filter( (col('date') >= lit('2021-03-24')) & (col('date') <= lit('2021-03-31')) )

new_covid = covid_2.join( df_1,  on=['location','new_cases'] , how='inner')
new_covid = new_covid.withColumn("new_cases", col("new_cases").cast(IntegerType()))

new_covid.select('date','location','new_cases').orderBy(col('new_cases').desc()).show(10)

+----------+-------------+---------+
|      date|     location|new_cases|
+----------+-------------+---------+
|2021-03-25|       Brazil|   100158|
|2021-03-24|United States|    86960|
|2021-03-31|        India|    72330|
|2021-03-24|       France|    65392|
|2021-03-31|       Turkey|    39302|
|2021-03-26|       Poland|    35145|
|2021-03-31|      Germany|    25014|
|2021-03-26|        Italy|    24076|
|2021-03-25|         Peru|    19206|
|2021-03-26|      Ukraine|    18226|
+----------+-------------+---------+
only showing top 10 rows



## SQL spark

In [ ]:
spark.sql('''select 
                b.date
                ,a.location
                ,a.new_cases
              from (select distinct
                      location 
                      ,max(new_cases)over(partition by location) as new_cases
                    from (select 
                            date
                            ,location 
                            ,cast(new_cases as int) as new_cases
                        from df_spark
                         where date >= '2021-03-24' and date <= '2021-03-31'
                            and location not in ('World','Europe','European Union','Asia','South America','North America','Africa')
                          )
                    ) a
              inner join 
                          (select 
                              date
                              ,location 
                              ,cast(new_cases as int) as new_cases
                            from df_spark
                            where date >= '2021-03-24' and date <= '2021-03-31'
                                and location not in ('World','Europe','European Union','Asia','South America','North America','Africa')
                          ) b 
              on a.location = b.location and a.new_cases = b.new_cases
              order by a.new_cases desc ;''').show(10)

+----------+-------------+---------+
|      date|     location|new_cases|
+----------+-------------+---------+
|2021-03-25|       Brazil|   100158|
|2021-03-24|United States|    86960|
|2021-03-31|        India|    72330|
|2021-03-24|       France|    65392|
|2021-03-31|       Turkey|    39302|
|2021-03-26|       Poland|    35145|
|2021-03-31|      Germany|    25014|
|2021-03-26|        Italy|    24076|
|2021-03-25|         Peru|    19206|
|2021-03-26|      Ukraine|    18226|
+----------+-------------+---------+
only showing top 10 rows



# 3. Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. (например: в россии вчера было 9150 , сегодня 8763, итог: -387) (в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)  
### *Pandas*

In [14]:
covid['yesterday'] = covid.loc[(covid['date']<='2021-03-31') & (covid['date']>='2021-03-24') & (covid['location'] == 'Russia'),['new_cases']]\
  .shift(periods=1, fill_value=0)
covid['diffirent'] = covid['new_cases']-covid['yesterday']
covid.loc[(covid['date']>='2021-03-25') & (covid['date']<='2021-03-31') & (covid['location'] == 'Russia'),\
          ['date','location','new_cases','yesterday','diffirent']]

,date,location,new_cases,yesterday,diffirent
61387,2021-03-25,Russia,9128.0,8769.0,359.0
61388,2021-03-26,Russia,9073.0,9128.0,-55.0
61389,2021-03-27,Russia,8783.0,9073.0,-290.0
61390,2021-03-28,Russia,8979.0,8783.0,196.0
61391,2021-03-29,Russia,8589.0,8979.0,-390.0
61392,2021-03-30,Russia,8162.0,8589.0,-427.0
61393,2021-03-31,Russia,8156.0,8162.0,-6.0


### PySpark

In [15]:
df_3 = df_spark.withColumn('yesterday', F.lag('new_cases',1).over(Window.partitionBy('location').orderBy('date')))\
.filter( (col('date') >= '2021-03-25') & (col('date') <= '2021-03-31') )

In [16]:
df_3 = df_3.withColumn('diffirent', col('new_cases') - col('yesterday'))

In [17]:
df_3.filter(col('location') == 'Russia').select('date','location','new_cases','yesterday','diffirent').show()

+----------+--------+---------+---------+---------+
|      date|location|new_cases|yesterday|diffirent|
+----------+--------+---------+---------+---------+
|2021-03-25|  Russia|   9128.0|   8769.0|    359.0|
|2021-03-26|  Russia|   9073.0|   9128.0|    -55.0|
|2021-03-27|  Russia|   8783.0|   9073.0|   -290.0|
|2021-03-28|  Russia|   8979.0|   8783.0|    196.0|
|2021-03-29|  Russia|   8589.0|   8979.0|   -390.0|
|2021-03-30|  Russia|   8162.0|   8589.0|   -427.0|
|2021-03-31|  Russia|   8156.0|   8162.0|     -6.0|
+----------+--------+---------+---------+---------+



## SQL spark

In [20]:
spark.sql('''select 
                b.date
                ,a.location
                ,a.new_cases
                ,yesterday
                ,b.new_cases - yesterday as diffirent
              from (select 
                        date
                        ,location
                        ,new_cases 
                        ,lag(new_cases)over(partition by location order by date) as yesterday
                    from df_spark
                    where date >='2021-03-24' and date <= '2021-03-31' and location = 'Russia'
                    ) a
              inner join df_spark b 
              on a.location = b.location and a.date = b.date
              where b.date >= '2021-03-25' and b.date <= '2021-03-31'
              order by date ;''').show(10)

+----------+--------+---------+---------+---------+
|      date|location|new_cases|yesterday|diffirent|
+----------+--------+---------+---------+---------+
|2021-03-25|  Russia|   9128.0|   8769.0|    359.0|
|2021-03-26|  Russia|   9073.0|   9128.0|    -55.0|
|2021-03-27|  Russia|   8783.0|   9073.0|   -290.0|
|2021-03-28|  Russia|   8979.0|   8783.0|    196.0|
|2021-03-29|  Russia|   8589.0|   8979.0|   -390.0|
|2021-03-30|  Russia|   8162.0|   8589.0|   -427.0|
|2021-03-31|  Russia|   8156.0|   8162.0|     -6.0|
+----------+--------+---------+---------+---------+

